## Optimización de parámetros con Redes Neuronales

In [ ]:
import pandas as pd

dataset_work_3 = pd.read_csv('D:\DataSet\dataset_work_transformed.csv')
x = dataset_work_3.iloc[:,:-1]
y = dataset_work_3.iloc[:,-1].values
dataset_work_3.shape

(6435823, 19)

In [ ]:
import numpy as np
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Registro del tiempo de inicio
start_time = time.time()

# 2. Separar las características (X) y la variable objetivo (y)
X = dataset_work_3.drop(columns=["MOROSIDAD"])
y = dataset_work_3["MOROSIDAD"]

# 3. Escalado de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. División del conjunto de datos en entrenamiento y prueba (sin PCA)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 5. Espacio de búsqueda reducido para los hiperparámetros
print("Espacio de búsqueda reducido para los hiperparámetros")
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant'],
    'max_iter': [100],
    'early_stopping': [True],
    'n_iter_no_change': [5],  # Menor número de iteraciones sin mejora
}

# 6. Inicializar el clasificador y RandomizedSearchCV
print("Inicializar el clasificador y RandomizedSearchCV")
mlp = MLPClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_dist,
    n_iter=5,  # Reducir el número de combinaciones
    cv=2,  # Validación cruzada con 2 folds
    n_jobs=1,  # Usar un solo núcleo para evitar problemas de memoria
    verbose=2,
    random_state=42,
    scoring='roc_auc'
)

# 7. Ajustar el modelo con la muestra del dataset
random_search.fit(X_train, y_train)

# 8. Obtener los mejores parámetros
print("Mejores parámetros encontrados:")
print(random_search.best_params_)

# 9. Entrenar el modelo final con todos los datos utilizando los mejores parámetros
best_params = random_search.best_params_
best_model = MLPClassifier(**best_params, random_state=42)

# Escalar el dataset completo (sin PCA)
X_full = dataset_work_3.drop(columns=["MOROSIDAD"])
y_full = dataset_work_3["MOROSIDAD"]
X_full_scaled = scaler.transform(X_full)

# División de los datos para el modelo final
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full_scaled, y_full, test_size=0.3, random_state=42)

# Ajustar el modelo con el dataset completo
best_model.fit(X_train_full, y_train_full)

# Hacer predicciones y evaluar el modelo
y_pred = best_model.predict(X_test_full)
y_pred_proba = best_model.predict_proba(X_test_full)[:, 1]

print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_full, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_full, y_pred))

roc_auc = roc_auc_score(y_test_full, y_pred_proba)
print(f"ROC AUC: {roc_auc:.2f}")

# Medir el tiempo de finalización y calcular el tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time

# Convertir a minutos y segundos
minutes = elapsed_time // 60
seconds = elapsed_time % 60

print(f"El tiempo total de ejecución fue de {int(minutes)} minutos y {seconds:.2f} segundos.")

Espacio de búsqueda reducido para los hiperparámetros
Inicializar el clasificador y RandomizedSearchCV
Fitting 2 folds for each of 5 candidates, totalling 10 fits
[CV] END activation=tanh, alpha=0.001, early_stopping=True, hidden_layer_sizes=(100,), learning_rate=constant, max_iter=100, n_iter_no_change=5, solver=adam; total time=20.2min
[CV] END activation=tanh, alpha=0.001, early_stopping=True, hidden_layer_sizes=(100,), learning_rate=constant, max_iter=100, n_iter_no_change=5, solver=adam; total time=10.9min
[CV] END activation=tanh, alpha=0.001, early_stopping=True, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=100, n_iter_no_change=5, solver=adam; total time=14.1min
[CV] END activation=tanh, alpha=0.001, early_stopping=True, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=100, n_iter_no_change=5, solver=adam; total time=10.4min
[CV] END activation=relu, alpha=0.0001, early_stopping=True, hidden_layer_sizes=(50,), learning_rate=constant, max_iter=100, n_iter